In [57]:
#conda install -c pytorch pytorch
import os
os.chdir('C:\\Users\\ABIARYAA\\Documents\\Downloads\\Deep learning A to Z\\DL Colab Changes\\Boltzmann_Machines 2')

In [60]:
#Import Packages
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable # FOR S.G.D

OSError: [WinError 126] The specified module could not be found

In [12]:
#Import dataset
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [17]:
print(movies.head(2))

   0                 1                             2
0  1  Toy Story (1995)   Animation|Children's|Comedy
1  2    Jumanji (1995)  Adventure|Children's|Fantasy


In [18]:
print(users.head(2))

   0  1   2   3      4
0  1  F   1  10  48067
1  2  M  56  16  70072


In [19]:
print(ratings.head(2)) # columns = ['user_id', 'movie_number', 'rating']

   0     1  2          3
0  1  1193  5  978300760
1  1   661  3  978302109


In [22]:
#Import train test sets
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set)
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set)

In [26]:
print(training_set)
print(training_set.shape)

[[        1         2         3 876893171]
 [        1         3         4 878542960]
 [        1         4         3 876893119]
 ...
 [      943      1188         3 888640250]
 [      943      1228         3 888640275]
 [      943      1330         3 888692465]]
(79999, 4)


In [27]:
print(test_set)
print(test_set.shape)

[[        1        10         3 875693118]
 [        1        12         5 878542960]
 [        1        14         5 874965706]
 ...
 [      459       934         3 879563639]
 [      460        10         3 882912371]
 [      462       682         5 886365231]]
(19999, 4)


In [31]:
#Getting the number of user and movies
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))  # columns = ['user_id', 'movie_number', 'rating']
nb_users = int(max(max(training_set[:,2]), max(test_set[:,2])))

In [33]:
print(nb_movies)
print(nb_users)

1682
5


In [54]:
#Convert data into an array wih users as rows and movies in columns with ratings in the cells
def convert(data):
    new_data = []
    for id_users in range(1,2 ):  #nb_users+1
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:, 2][data[:,0]== id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies-1] = id_ratings
        new_data.append(list(ratings))   #List of lists which torch expects
    return new_data

In [55]:
training_set = convert(training_set)
test_set = convert(test_set)

In [ ]:
#Convert data into torch tensors
training_set = torch.FloatTensor(training_set)  # multi dim float array
test_set = torch.FloatTensor(test_set)

In [ ]:
#Converting the ratings ino binary ratings 1 or 0 for like/dislike
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1

test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >=3] = 1

In [ ]:
class RBM():
    def _init_(self, nv, nh):
        self.W = torch.randn(nh,nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1,nv)
    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    def train(self, v0, vk, ph0, phk):
        self.W += torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)
        self.b += torch.sum((v0-vk), 0)
        self.a += torch.sum((ph0-phk), 0)